In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('always')
from sklearn.model_selection import train_test_split
import shapely.speedups
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
import re
from itertools import combinations
import random
import joblib

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
#
np.random.seed(1)
tf.random.set_seed(1)

c:\Users\iankl\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:18: DeprecationWarning: Call to deprecated create function FileDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  DESCRIPTOR = _descriptor.FileDescriptor(
c:\Users\iankl\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:36: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.FieldDescriptor(
c:\Users\iankl\anaconda3\lib\site-packages\tensorflow\core\framework\tensor_shape_pb2.py:43: DeprecationWarning: Call to deprecated create function FieldDescriptor(). Note: Create unlinked descriptors is going to go away. Please use get/find descriptors from generated code or query the descriptor_pool.
  _descriptor.F

In [3]:
def plot_regression(modelo,x,y,title="",xlabel="x",ylabel="y"):
    plt.figure()
    
    plt.plot(x,y,"o",label="Valores verdaderos")
    plt.plot(x,modelo.predict(x),"x",label="Valores estimados")
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [4]:
viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_test.csv').copy()
viviendas = pd.concat([viviendas_train, viviendas_test])

# regresion

Realizamos un one hot encoding para poder utilizar el barrio y type para regresion

In [5]:
viviendas = pd.get_dummies(viviendas, columns = ["property_type"])
viviendas = pd.get_dummies(viviendas, columns = ["barrio"])

definimos las columnas que utilizaremos para entrenar las redes

In [6]:
columnas_predictoras=viviendas.columns.to_list()
for variable in ['property_price', 'start_date', 'end_date', 'property_title', 'geometry']:
    columnas_predictoras.remove(variable)
columnas_predictoras

['latitud',
 'longitud',
 'property_rooms',
 'property_bedrooms',
 'property_surface_total',
 'property_surface_covered',
 'property_type_Casa',
 'property_type_Departamento',
 'property_type_PH',
 'barrio_Abasto',
 'barrio_Agronomía',
 'barrio_Almagro',
 'barrio_Balvanera',
 'barrio_Barracas',
 'barrio_Barrio Norte',
 'barrio_Belgrano',
 'barrio_Boca',
 'barrio_Boedo',
 'barrio_Caballito',
 'barrio_Catalinas',
 'barrio_Centro / Microcentro',
 'barrio_Chacarita',
 'barrio_Coghlan',
 'barrio_Colegiales',
 'barrio_Congreso',
 'barrio_Constitución',
 'barrio_Flores',
 'barrio_Floresta',
 'barrio_Las Cañitas',
 'barrio_Liniers',
 'barrio_Mataderos',
 'barrio_Monserrat',
 'barrio_Monte Castro',
 'barrio_Nuñez',
 'barrio_Once',
 'barrio_Palermo',
 'barrio_Parque Avellaneda',
 'barrio_Parque Centenario',
 'barrio_Parque Chacabuco',
 'barrio_Parque Chas',
 'barrio_Parque Patricios',
 'barrio_Paternal',
 'barrio_Pompeya',
 'barrio_Puerto Madero',
 'barrio_Recoleta',
 'barrio_Retiro',
 'barrio_S

## RANDOMIZADOR


Para ontrenar las redes neuronales utilizaresmos un random_search para optimizar los hiperparametros

In [7]:
#al instanciar un ModeloRedes crea y entrena una red neuronal con los hiperparametros pasados
class ModeloRedes:
    def __init__(self, dataset, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size, cantidad_filas):
        self.variables_predictoras = variables_predictoras
        self.cantidad_primera_capa = cantidad_de_primera_capa
        self.funcion_activacion = funcion_activacion
        self.optimizador = optimizador
        self.learning_rate = learning_rate
        self.resultado = None
        self.metrica = metrica
        self.epoch = epoch
        self.batch_size = batch_size
        self.modelo = None

        
        self.x_test = None
        self.y_test = None
        self.x_train = None
        self.y_train = None
        self.dataset_trabajo = dataset[:cantidad_filas]
        self.precio_maximo = max(self.dataset_trabajo['property_price'])
        self.obtener_datasets()
        self.entrenar_modelo()

    def obtener_datasets(self):    
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.dataset_trabajo.loc[:,self.variables_predictoras],self.dataset_trabajo.loc[:,'property_price'],test_size=0.2)
        self.escalar_datasets()
    
    def escalar_datasets(self):
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(self.x_train))
        self.x_train=sscaler.transform(pd.DataFrame(self.x_train))
        self.x_test=sscaler.transform(pd.DataFrame(self.x_test))
            
        self.y_train = self.y_train / self.precio_maximo
        

    def entrenar_modelo(self):
    # Creo un modelo Red Neuronal
        d_in= len(self.variables_predictoras)
        d_out=1

        capas = [
            # input layer
            keras.layers.Dense(d_in,input_shape=(d_in,), activation= self.funcion_activacion)
        ]

        #inserta capas que van decreciendo en cantidad de neuronas
        for i in range(self.cantidad_primera_capa,2,-1):
            capas.append(keras.layers.Dense(i, activation= self.funcion_activacion ))

        capas.append(keras.layers.Dense(d_out, ))
        self.modelo = keras.Sequential(capas)

        self.modelo.compile(
        optimizer= self.optimizador(self.learning_rate), 
        loss=self.metrica, 
        metrics=[self.metrica], 
        )

        self.modelo.fit(self.x_train,self.y_train,epochs=self.epoch,batch_size=self.batch_size,verbose=False)
    
    def plotear(self):
        plot_regression(self.modelo,self.x_test,self.y_test,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        
    def testear_modelo(self):
        y_pred=self.modelo.predict(self.x_test)
        for i in y_pred:
            i[0] = i[0] * self.precio_maximo
        mse=mean_squared_error(self.y_test,y_pred)
        self.resultado = mse ** (1/2)
        return self.resultado
    
    def predecir(self, df):

        df_para_predecir =  df.loc[:,self.variables_predictoras]
        df_precios = df.loc[:,'property_price']
        
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(df_para_predecir))
        df_para_predecir=sscaler.transform(pd.DataFrame(df_para_predecir))

        
        y_pred=self.modelo.predict(df_para_predecir)
        for i in y_pred:
            i[0] = i[0] * self.precio_maximo

        mse=mean_squared_error(df_precios,y_pred)
        resultado = mse ** (1/2)
        #plot_regression(self.modelo,df_para_predecir,df_precios,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        return resultado
    
    def info(self):
        print('\n','\n')
        print('\n',
        self.variables_predictoras , '\n',
        self.cantidad_primera_capa ,'\n',
        self.funcion_activacion ,'\n',
        self.optimizador ,'\n',
        self.learning_rate ,'\n',
        self.metrica ,'\n',
        self.epoch ,'\n',
        self.batch_size ,'\n',
        self.dataset_trabajo.shape ,'\n')

definimos los conjuntos de hiperparametros que queremos que pruebe

In [8]:
columnas_importantes =  ['latitud', 'longitud', 'property_surface_total', 'property_surface_covered']
columnas_extras = []
for e in columnas_predictoras:
    if e not in columnas_importantes:
        columnas_extras.append(e)       

def agregar_columnas(posibles_variables_predictoras, columnas, cant_columnas):
            i = 0
            while (i < cant_columnas): 
                pos = random.randrange(i,len(columnas), 1)
                posibles_variables_predictoras.append(columnas[pos])
                columnas[i], columnas[pos] = columnas[pos], columnas[i]
                i += 1

posibles_cantidad_de_primera_capa = []
for i in range(1,20):
    posibles_cantidad_de_primera_capa.append(i)
                                        #vimos que los modelos buenos utilizaban "relu" asique dejamos de utilizar el resto dentro de random search
posibles_funcion_activacion = ["relu","tanh", "softmax", "sigmoid"]
posibles_metricas = ['mse', 'mae']

posibles_learning_rate = [0.1, 0.01]

In [9]:
def funcion_optimizadoraNADAM(learnigRate):
            return keras.optimizers.Nadam(learning_rate = learnigRate)

def funcion_optimizadoraRMSPROP(learnigRate):
            return keras.optimizers.RMSprop(learning_rate = learnigRate)

def funcion_optimizadoraADAMAX(learnigRate):
            return keras.optimizers.Adamax(learning_rate = learnigRate)

def funcion_optimizadoraADADELTA(learnigRate):
            return keras.optimizers.Adadelta(learning_rate = learnigRate)

posibles_optimizador = [funcion_optimizadoraNADAM, funcion_optimizadoraRMSPROP, funcion_optimizadoraADAMAX, funcion_optimizadoraADADELTA]

In [10]:
def obtener_posibles_variables_predictoras():
        posibles_variables_predictoras = []     
        cant_columnas_importantes = random.randrange(1,len(columnas_importantes)+1, 1)
        cant_columnas_extras = random.randrange(0,len(columnas_extras)+1, 1)

        agregar_columnas(posibles_variables_predictoras, columnas_importantes, cant_columnas_importantes)
        agregar_columnas(posibles_variables_predictoras, columnas_extras, cant_columnas_extras)
        return posibles_variables_predictoras

crea, entrena y luego predice un modelo con hiperparametros randomizados sobre datasets peque;os para reducir el tiempo de computo y obtener buenos hiperparametros

In [11]:
def random_validator(cantidad_modelos, columnas_importantes, columnas_extras, posibles_cantidad_de_primera_capa, 
    posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas, obtener_posibles_variables_predictoras):
    resultados = []
    for i in range(cantidad_modelos):

        variables_predictoras = obtener_posibles_variables_predictoras()
        cantidad_de_primera_capa = random.choice(posibles_cantidad_de_primera_capa)
        funcion_activacion = random.choice(posibles_funcion_activacion)
        optimizador = random.choice(posibles_optimizador)
        learning_rate = random.choice(posibles_learning_rate)
        metrica = random.choice(posibles_metricas)
        epoch = random.randrange(100, 300, 10)
        batch_size = random.randrange(10, 1000, 1)
        cantidad_filas = 3000#random.randrange(3000, 5000, 1)

        print('\n','\n')
        print(i , '\n',
        variables_predictoras , '\n',
        cantidad_de_primera_capa ,'\n',
        funcion_activacion ,'\n',
        optimizador ,'\n',
        learning_rate ,'\n',
        metrica ,'\n',
        epoch ,'\n',
        batch_size ,'\n',
        cantidad_filas ,'\n')

        modelo = ModeloRedes(viviendas, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size, cantidad_filas)
        resultados.append([modelo, modelo.testear_modelo()])
        print(modelo.testear_modelo())
    return resultados

In [12]:
#Hace 50 modelos con hiperparametros random random
def func():
    resultados = random_validator(50,columnas_importantes, columnas_extras, posibles_cantidad_de_primera_capa, posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas,obtener_posibles_variables_predictoras)

    return resultados

In [13]:
#dado un conjunto de modelos se queda con los modelos cuyo rmse sea menor a 130000
def obtener_mejores(resultados):
    mejores = []
    for i in resultados:
        if (i[1] < 130000):
            mejores.append(i)

    return mejores

Entrenamos 150 modelos

In [14]:
mejores = []

In [15]:

resultados = func()
mejores.append(obtener_mejores(resultados))


 

0 
 ['longitud', 'barrio_Villa General Mitre', 'barrio_Parque Avellaneda', 'barrio_Las Cañitas', 'barrio_Coghlan', 'property_type_Departamento', 'property_type_Casa', 'barrio_Congreso', 'barrio_Agronomía', 'barrio_Boedo', 'barrio_Palermo', 'barrio_Parque Centenario', 'barrio_Once', 'barrio_Retiro', 'barrio_Balvanera', 'barrio_Abasto', 'barrio_Puerto Madero', 'barrio_Villa Santa Rita', 'barrio_Centro / Microcentro', 'barrio_Tribunales', 'barrio_Liniers', 'barrio_Parque Chacabuco', 'property_type_PH', 'barrio_Villa Real', 'barrio_Catalinas', 'barrio_Villa Ortuzar', 'barrio_Parque Patricios', 'barrio_San Telmo', 'barrio_Villa Riachuelo', 'barrio_Caballito', 'barrio_Villa Pueyrredón', 'barrio_Barrio Norte', 'property_bedrooms', 'property_rooms', 'barrio_Barracas', 'barrio_Constitución', 'barrio_Boca', 'barrio_Villa Luro', 'barrio_Saavedra', 'barrio_Mataderos', 'barrio_Villa Crespo', 'barrio_Chacarita', 'barrio_Pompeya', 'barrio_Flores', 'barrio_Colegiales', 'barrio_Monte Castro', 'barr

In [16]:

resultados = func()
mejores.append(obtener_mejores(resultados))


 

0 
 ['longitud', 'latitud', 'property_surface_covered', 'barrio_San Cristobal', 'barrio_Tribunales', 'barrio_Mataderos', 'barrio_Belgrano', 'barrio_Villa Lugano', 'barrio_Villa Real', 'barrio_Barracas', 'barrio_Villa Devoto', 'barrio_San Nicolás', 'barrio_Villa Ortuzar', 'barrio_Villa Luro', 'barrio_Flores', 'barrio_Almagro', 'property_bedrooms', 'barrio_Parque Chas', 'barrio_Villa Santa Rita', 'barrio_Palermo', 'barrio_Parque Chacabuco', 'barrio_Nuñez', 'barrio_Parque Avellaneda', 'barrio_Liniers', 'barrio_Coghlan', 'barrio_Villa del Parque', 'barrio_Boedo', 'barrio_Centro / Microcentro', 'barrio_Constitución', 'barrio_Las Cañitas', 'barrio_Balvanera', 'barrio_Paternal', 'barrio_Puerto Madero', 'barrio_Once', 'barrio_Congreso', 'barrio_Boca', 'barrio_Agronomía', 'barrio_Retiro', 'barrio_Recoleta', 'barrio_Catalinas', 'property_type_PH', 'barrio_Velez Sarsfield', 'barrio_Saavedra', 'barrio_Chacarita', 'barrio_Villa Pueyrredón', 'barrio_San Telmo', 'barrio_Monserrat', 'barrio_Flores

In [17]:

resultados = func()
mejores.append(obtener_mejores(resultados))


 

0 
 ['property_surface_total', 'barrio_Velez Sarsfield', 'barrio_Pompeya', 'barrio_Villa Pueyrredón', 'barrio_Versalles', 'barrio_Almagro', 'barrio_San Cristobal', 'barrio_Villa Lugano', 'barrio_Villa General Mitre', 'barrio_Floresta', 'barrio_Parque Chas', 'barrio_Caballito', 'property_type_PH', 'property_bedrooms', 'barrio_Barrio Norte', 'barrio_Congreso', 'barrio_Puerto Madero', 'barrio_Villa Urquiza', 'barrio_Coghlan', 'barrio_Villa Crespo', 'barrio_Villa Real', 'barrio_Villa del Parque', 'barrio_Colegiales', 'barrio_Parque Avellaneda', 'barrio_Constitución', 'barrio_Mataderos', 'barrio_Nuñez'] 
 13 
 tanh 
 <function funcion_optimizadoraADADELTA at 0x000001AF336BDB80> 
 0.1 
 mse 
 220 
 398 
 3000 

19/19 [==============================] - 0s 623us/step
188732.77405707954

 

1 
 ['property_surface_total', 'longitud', 'property_surface_covered', 'latitud', 'barrio_Villa Real', 'barrio_Puerto Madero', 'barrio_Almagro', 'barrio_Villa Soldati', 'barrio_Villa Lugano', 'barrio_Boe

In [18]:

resultados = func()
mejores.append(obtener_mejores(resultados))


 

0 
 ['latitud', 'longitud', 'property_surface_total', 'property_surface_covered', 'barrio_Abasto', 'barrio_San Cristobal', 'barrio_Monserrat', 'barrio_Once', 'barrio_Saavedra', 'barrio_Parque Patricios', 'barrio_Balvanera', 'barrio_Tribunales', 'barrio_Villa Riachuelo', 'barrio_Constitución', 'barrio_Villa del Parque', 'barrio_Las Cañitas', 'barrio_Congreso', 'barrio_Villa Pueyrredón', 'barrio_Recoleta', 'barrio_Villa Real', 'property_type_Casa', 'barrio_Parque Centenario', 'barrio_Villa Ortuzar', 'barrio_Boedo', 'barrio_Mataderos', 'barrio_Parque Chas', 'barrio_Nuñez', 'barrio_Parque Chacabuco', 'barrio_Villa Santa Rita', 'barrio_Monte Castro', 'barrio_Chacarita', 'barrio_Floresta', 'property_bedrooms', 'barrio_Almagro', 'barrio_Versalles', 'barrio_Palermo', 'barrio_Velez Sarsfield', 'barrio_Villa Luro', 'property_type_PH', 'property_rooms', 'barrio_San Telmo', 'barrio_Caballito', 'barrio_Villa Devoto', 'barrio_Barrio Norte', 'property_type_Departamento'] 
 17 
 softmax 
 <functio

In [19]:

resultados = func()
mejores.append(obtener_mejores(resultados))


 

0 
 ['longitud', 'latitud', 'barrio_Balvanera', 'barrio_Velez Sarsfield', 'barrio_Villa Pueyrredón', 'barrio_Villa Luro', 'barrio_Monserrat', 'barrio_Versalles', 'barrio_Villa del Parque', 'barrio_Retiro', 'barrio_Villa Ortuzar', 'barrio_Pompeya', 'barrio_Chacarita', 'barrio_Barracas', 'barrio_Puerto Madero', 'barrio_Saavedra', 'barrio_Tribunales', 'barrio_Parque Centenario', 'barrio_Barrio Norte', 'barrio_Villa Urquiza', 'barrio_Villa General Mitre', 'barrio_Constitución', 'barrio_San Telmo', 'barrio_Paternal', 'barrio_Mataderos', 'barrio_Villa Riachuelo', 'barrio_Nuñez', 'barrio_Villa Santa Rita', 'barrio_Flores', 'barrio_Caballito', 'property_bedrooms', 'barrio_Centro / Microcentro', 'barrio_Congreso', 'barrio_Boedo', 'barrio_San Nicolás', 'barrio_Once', 'property_type_Casa', 'barrio_Catalinas', 'barrio_Parque Patricios', 'barrio_Floresta', 'barrio_Villa Devoto'] 
 6 
 sigmoid 
 <function funcion_optimizadoraRMSPROP at 0x000001AF336BDA60> 
 0.01 
 mse 
 280 
 341 
 3000 

19/19 

In [20]:
for i in mejores:
    for j in i:
        print(j)
        j[0].info()

[<__main__.ModeloRedes object at 0x000001AF800AF040>, 109182.58486885276]

 


 ['latitud', 'property_surface_covered', 'property_surface_total', 'longitud', 'property_type_PH', 'property_rooms', 'barrio_Versalles', 'barrio_Boca', 'barrio_Monte Castro', 'barrio_Tribunales', 'barrio_Centro / Microcentro', 'barrio_Villa Crespo', 'barrio_Velez Sarsfield', 'barrio_Colegiales', 'barrio_Once', 'barrio_Villa Santa Rita', 'property_type_Casa', 'barrio_Floresta', 'barrio_Villa Soldati', 'barrio_Parque Chas', 'barrio_Villa General Mitre', 'barrio_Palermo', 'barrio_Saavedra', 'barrio_Constitución', 'barrio_Mataderos', 'barrio_Boedo', 'barrio_Parque Chacabuco', 'barrio_Congreso', 'barrio_Caballito', 'barrio_Barracas', 'barrio_Monserrat', 'barrio_Puerto Madero', 'barrio_Villa Ortuzar', 'barrio_Barrio Norte', 'property_bedrooms', 'barrio_San Telmo', 'barrio_Villa del Parque', 'barrio_Villa Luro', 'barrio_San Nicolás'] 
 11 
 relu 
 <function funcion_optimizadoraNADAM at 0x000001AF336BD280> 
 0.01 
 

Entrenamos modelos con los hiperparametros obtenidos en los datasets mas chicos sobre todo el dataset

In [21]:
#reentrena un modelo con los mismo hiperparametros a ver si con una semilla mejor mejora la metrica
def reentrenar_tot(modelo):
    #modelo.info()
    mejor_modelo = ModeloRedes(viviendas, modelo.variables_predictoras, modelo.cantidad_primera_capa, modelo.funcion_activacion, modelo.optimizador, modelo.learning_rate, modelo.metrica, modelo.epoch, modelo.batch_size, viviendas.size)
    mejor_modelo.testear_modelo()
    mejor_resultado = mejor_modelo.resultado 
   
    if mejor_resultado > 150000:
        return None

    for i in range(2):
        print('valor a vencer', mejor_resultado)
        modelo_aux = ModeloRedes(viviendas, modelo.variables_predictoras, modelo.cantidad_primera_capa, modelo.funcion_activacion, modelo.optimizador, modelo.learning_rate, modelo.metrica, modelo.epoch, modelo.batch_size, viviendas.size)
        resultado_aux = modelo_aux.testear_modelo()
        print('reentrenando intento', i+1)
        resultado_aux = modelo_aux.testear_modelo()
        print(resultado_aux)
        mejor_modelo.info()
        
        if resultado_aux < mejor_resultado:
            mejor_resultado = resultado_aux
            mejor_modelo = modelo_aux 
    return [mejor_modelo, mejor_resultado]

In [22]:
tots = []



In [23]:
for i in mejores:
    for j in i:
        tots.append(reentrenar_tot(j[0]))

581/581 [==============================] - 1s 767us/step
valor a vencer 111028.45091946187
581/581 [==============================] - 1s 765us/step
reentrenando intento 1
581/581 [==============================] - 0s 767us/step
108163.4798774882

 


 ['latitud', 'property_surface_covered', 'property_surface_total', 'longitud', 'property_type_PH', 'property_rooms', 'barrio_Versalles', 'barrio_Boca', 'barrio_Monte Castro', 'barrio_Tribunales', 'barrio_Centro / Microcentro', 'barrio_Villa Crespo', 'barrio_Velez Sarsfield', 'barrio_Colegiales', 'barrio_Once', 'barrio_Villa Santa Rita', 'property_type_Casa', 'barrio_Floresta', 'barrio_Villa Soldati', 'barrio_Parque Chas', 'barrio_Villa General Mitre', 'barrio_Palermo', 'barrio_Saavedra', 'barrio_Constitución', 'barrio_Mataderos', 'barrio_Boedo', 'barrio_Parque Chacabuco', 'barrio_Congreso', 'barrio_Caballito', 'barrio_Barracas', 'barrio_Monserrat', 'barrio_Puerto Madero', 'barrio_Villa Ortuzar', 'barrio_Barrio Norte', 'property_bedrooms', 

In [24]:
print(tots)

[[<__main__.ModeloRedes object at 0x000001AFA98AD730>, 108163.4798774882], [<__main__.ModeloRedes object at 0x000001AFD14D7D30>, 125473.06252974333], None, [<__main__.ModeloRedes object at 0x000001AF957A6880>, 119208.515736817], [<__main__.ModeloRedes object at 0x000001AFE1E00E20>, 129338.1455767638], [<__main__.ModeloRedes object at 0x000001AF9D300F10>, 120680.80781551782], None, [<__main__.ModeloRedes object at 0x000001AFBBD26EB0>, 119577.83407628175]]


In [25]:
mejor_modelo = tots[0]
for i in tots:
    if (i != None) and (i[1] < mejor_modelo[1]):
        mejor_modelo = i



In [26]:
mejor_modelo[0].testear_modelo()

581/581 [==============================] - 1s 928us/step


108163.4798774882

In [27]:
joblib.dump(mejor_modelo[0], './Models/mejor_modelo_ultima_tanda') 

INFO:tensorflow:Assets written to: ram://c7ac60f2-b084-4fcd-9d9b-dca2c704736b/assets


['./Models/mejor_modelo_ultima_tanda']

El mejor modelo obtenido hasta ahora con estas tecnicas es:

In [28]:
mejor_modelo = joblib.load('./Models/modelo_regresion_103k')

ReadError: file could not be opened successfully

In [29]:
print('Metrica rmse', mejor_modelo.testear_modelo())

mejor_modelo.info()

AttributeError: 'list' object has no attribute 'testear_modelo'

In [ ]:
#joblib.dump(model_tot, './Models/red_regresion_108k_tot') 